In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore') 
import pandas as pd
import numpy as np
from sklearn.linear_model import RANSACRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from time import time
import joblib

In [2]:
df = pd.read_excel('/kaggle/input/prediksisiklusmenstruasi/dataset_v5.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ClientID                  149 non-null    object 
 1   LengthofCycle             149 non-null    int64  
 2   EstimatedDayofOvulation   149 non-null    float64
 3   LengthofMenses            149 non-null    int64  
 4   Age                       149 non-null    int64  
 5   BMI                       149 non-null    float64
 6   CycleNumber               149 non-null    int64  
 7   LengthofCycle1            149 non-null    int64  
 8   LengthofCycle2            149 non-null    int64  
 9   LengthofCycle3            149 non-null    int64  
 10  LengthOfMenses1           149 non-null    int64  
 11  LengthOfMenses2           149 non-null    int64  
 12  LengthOfMenses3           149 non-null    int64  
 13  MeanOfMenses              149 non-null    float64
 14  EstimatedD

In [3]:
df.head()

,ClientID,LengthofCycle,EstimatedDayofOvulation,LengthofMenses,Age,BMI,CycleNumber,LengthofCycle1,LengthofCycle2,LengthofCycle3,LengthOfMenses1,LengthOfMenses2,LengthOfMenses3,MeanOfMenses,EstimatedDayofOvulation1,EstimatedDayofOvulation2,EstimatedDayofOvulation3
0,nfp8020,28,14.0,5,33,25.790533,4,27,31,27,6,6,7,6.333333,13,17,15
1,nfp8024,26,15.0,5,23,24.958580,4,29,30,28,5,5,5,5.000000,16,18,15
2,nfp8026,27,16.0,6,39,21.454890,4,26,30,25,5,10,5,6.666667,15,19,11
3,nfp8030,25,12.0,6,23,22.462722,4,26,29,29,6,5,5,5.333333,13,16,14
4,nfp8031,27,13.0,6,30,21.948577,4,28,27,27,7,5,6,6.000000,14,15,13


In [4]:
df = df.drop(['ClientID','EstimatedDayofOvulation','LengthofMenses','CycleNumber','MeanOfMenses','EstimatedDayofOvulation1','EstimatedDayofOvulation2','EstimatedDayofOvulation3'	], axis=1)

In [5]:
df.head()

,LengthofCycle,Age,BMI,LengthofCycle1,LengthofCycle2,LengthofCycle3,LengthOfMenses1,LengthOfMenses2,LengthOfMenses3
0,28,33,25.790533,27,31,27,6,6,7
1,26,23,24.958580,29,30,28,5,5,5
2,27,39,21.454890,26,30,25,5,10,5
3,25,23,22.462722,26,29,29,6,5,5
4,27,30,21.948577,28,27,27,7,5,6


In [6]:
# Split dataset

X = df.drop(['LengthofCycle'], axis=1)
y = df['LengthofCycle']

In [7]:
# Membagi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Regression Modelling

In [8]:
regresi = [RANSACRegressor(random_state=42),
           DecisionTreeRegressor(random_state=42),
           RandomForestRegressor(random_state=42)]

In [9]:
head = 4
for model in regresi[:head]:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    predict_time = time()-start
    val_sil = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    score_val_sil = -val_sil
    print(model)
    print("\tWaktu latih: %0.3fs" % train_time)
    print("\tWaktu prediksi: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, y_pred))
    print("\tMSE:", mean_squared_error(y_test, y_pred))
    print("\tR2 score:", r2_score(y_test, y_pred))
    print('\tMSE validasi silang:', score_val_sil)
    print('\tRata-rata MSE validasi silang:', np.mean(score_val_sil))
    print()

RANSACRegressor(random_state=42)
	Waktu latih: 0.142s
	Waktu prediksi: 0.002s
	Explained variance: 0.13363144917202052
	MSE: 13.744511495783188
	R2 score: 0.016766525220183848
	MSE validasi silang: [16.77739474 22.21768413  8.49900672 16.44974092 28.56011676]
	Rata-rata MSE validasi silang: 18.500788654853586

DecisionTreeRegressor(random_state=42)
	Waktu latih: 0.004s
	Waktu prediksi: 0.002s
	Explained variance: -0.3138065336618707
	MSE: 19.3
	R2 score: -0.38065336618710743
	MSE validasi silang: [20.83333333 17.45833333 12.33333333 17.45833333 46.65217391]
	Rata-rata MSE validasi silang: 22.94710144927536

RandomForestRegressor(random_state=42)
	Waktu latih: 0.179s
	Waktu prediksi: 0.006s
	Explained variance: 0.22487595580637487
	MSE: 11.474306666666667
	R2 score: 0.17916890549240938
	MSE validasi silang: [11.74213333 10.51611667  6.06707083  8.10315833 27.68138696]
	Rata-rata MSE validasi silang: 12.821973224637683



In [10]:
parameter = {'criterion':['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
             'max_features':['sqrt', 'log2', None],
             'bootstrap':[True, False]
             }

grid = GridSearchCV(RandomForestRegressor(random_state=42), parameter)
model = grid.fit(X, y)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

{'bootstrap': True, 'criterion': 'absolute_error', 'max_features': 'sqrt'} 

RandomForestRegressor(criterion='absolute_error', max_features='sqrt',
                      random_state=42) 



In [11]:
rfr = RandomForestRegressor(criterion='absolute_error',max_features='sqrt', random_state=42)
rfr.fit(X_train, y_train)

RandomForestRegressor(criterion='absolute_error', max_features='sqrt',
                      random_state=42)

In [12]:
# model keras

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import Adam, RMSprop, SGD, Nadam, Adagrad
from keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(units=512, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='linear'))

# Compile the model
model.compile(optimizer=Nadam(learning_rate=0.0001), loss=MeanSquaredError(), metrics=['mse'])


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1, validation_data=(X_test, y_test), callbacks=[early_stopping])

#model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1, validation_data=(X_test, y_test))


Epoch 1/100
8/8 [==============================] - 7s 35ms/step - loss: 970.4485 - mse: 970.4485 - val_loss: 840.9426 - val_mse: 840.9426
Epoch 2/100
8/8 [==============================] - 0s 8ms/step - loss: 752.3350 - mse: 752.3350 - val_loss: 659.9354 - val_mse: 659.9354
Epoch 3/100
8/8 [==============================] - 0s 8ms/step - loss: 584.4892 - mse: 584.4892 - val_loss: 493.2137 - val_mse: 493.2137
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 423.9703 - mse: 423.9703 - val_loss: 332.1047 - val_mse: 332.1047
Epoch 5/100
8/8 [==============================] - 0s 8ms/step - loss: 270.4314 - mse: 270.4314 - val_loss: 180.9039 - val_mse: 180.9039
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 134.1063 - mse: 134.1063 - val_loss: 71.1580 - val_mse: 71.1580
Epoch 7/100
8/8 [==============================] - 0s 8ms/step - loss: 50.1951 - mse: 50.1951 - val_loss: 20.3918 - val_mse: 20.3918
Epoch 8/100
8/8 [=============================

In [15]:
# Save the sklearn model
joblib.dump((rfr), 'prediksimenstruasi.model')
import pickle
with open('prediksimenstruasi.pkl', 'wb') as model_file:
    pickle.dump(rfr, model_file)
joblib.dump((rfr), 'prediksimenstruasi.model')


# save the kears keras
model.save('prediksimenstruasi.h5')

In [19]:
import joblib
import pandas as pd


from keras.models import load_model

# load the model from sklearn model
loaded_model = joblib.load('prediksimenstruasi.model')

# Load the model from the HDF5 file
loaded_model1 = load_model('prediksimenstruasi.h5')

# New Data to Predict
new_data = pd.DataFrame(data={
    'Age': [33],
    'BMI': [25.790533],
    'LengthofCycle1': [27],
    'LengthofCycle2': [31],
    'LengthofCycle3': [27],
    'LengthOfMenses1': [6],
    'LengthOfMenses2': [6],
    'LengthOfMenses3': [7],
})

# Predict
predictions = loaded_model.predict(new_data)
predictions1 = loaded_model1.predict(new_data)

# Show the results of prediction
print('Prediksi Length of cycle (Sklearn) adalah:', predictions[0])
print('Prediksi Length of cycle (Keras) adalah:', predictions1[0])

1/1 [==============================] - 0s 63ms/step
Prediksi Length of cycle (Sklearn) adalah: 28.47
Prediksi Length of cycle (Keras) adalah: [28.228853]


In [20]:
# Convert the Keras model to TensorFlow.js format
!tensorflowjs_converter --input_format=keras /kaggle/working/prediksihaid.h5 /kaggle/working/tfjs_model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
